In [1]:
# Google driveと接続
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install japanize-matplotlib
# 必要なパッケージのインポート
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize
import japanize_matplotlib  # 日本語対応のフォントパッケージ
import os
from matplotlib.backends.backend_pdf import PdfPages

# 作業ディレクトリを変更（Google Drive内の'musashino'フォルダに移動）
os.chdir('/content/drive/MyDrive/musashino')

# 保存されたシェープファイル（GISデータ）を読み込み
Geo_musashino = gpd.read_file("geo_musashino.shp")

# CSVファイルが保存されているディレクトリを指定
directory = '/content/drive/MyDrive/musashino'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=139962d0e4613615b477b1ff45765bfc82a0517af2486c49e0ef772cfa5b350d
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize-matplotlib


In [3]:
# 地図を作成する関数を定義
def plot_musashino_map(num, ax):
    geo_musashino = Geo_musashino
    species = pd.read_csv(csv_files[num-1], encoding='cp932')

    # カラム名を変更、数値データに変換
    species.rename(columns={'町丁目': 'S_NAME'}, inplace=True)
    species['数'] = species['数'].replace('複', 2).astype(float, errors='ignore')
    # '数' カラムの欠損値を 0 に置き換える
    species['数'] = species['数'].fillna(0)

    # データ集計と町名の標準化関数
    def standardize(name):
        return name.replace('一丁目', '１丁目').replace('二丁目', '２丁目').replace('三丁目', '３丁目')\
                   .replace('四丁目', '４丁目').replace('五丁目', '５丁目')

    # 標準化と結合
    species_summary = species.groupby('S_NAME')['数'].sum().reset_index()
    geo_musashino = geo_musashino.merge(species_summary, on='S_NAME', how='left').fillna(0)

    # 色と地図描画
    cmap = LinearSegmentedColormap.from_list("custom_green", ["white", "green"], N=256)
    geo_musashino.plot(ax=ax, column='数', edgecolor='black', cmap=cmap, norm=Normalize(vmin=0, vmax=geo_musashino['数'].max()))

    # 中心に町名を表示
    geo_musashino.apply(lambda row: ax.annotate(row['S_NAME'], xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                                                ha='center', fontsize=9), axis=1)

    # 重要地点の座標と色
    locations = {
        '武蔵野市役所': (35.718678536303486, 139.56605171529222, 'red'),
        '成蹊大学': (35.71397106285028, 139.57389642384348, 'blue'),
        '武蔵野市中央公園': (35.71878088518305, 139.55860937155464, 'yellow'),
        '井の頭自然文化園': (35.70172742930674, 139.57269271009636, 'orange')
    }

    # 各場所にマーカーを追加
    for name, (lat, lon, color) in locations.items():
        ax.scatter(lon, lat, color=color, marker='o', s=100, label=name)

    # タイトル、ラベル、装飾
    ax.set_title(f"{csv_files[num-1][2:-4]}マップ", bbox=dict(facecolor='white', alpha=0.5, edgecolor='black'))
    ax.legend(loc='lower center', bbox_to_anchor=(0.475, 0.1), ncol=1)

    # 軸や枠線の完全削除
    ax.set_axis_off()  # これにより全ての枠線と軸が非表示になる

    # テキスト追加
    ax.text(0.33, 0.8, 'むさしのサイエンスフェスタ2024', fontsize=12, ha='right', va='top', transform=ax.transAxes)


In [4]:
# 武蔵野市生き物リストの表示
csv_files = [file for file in os.listdir() if file.endswith('.csv')]
# CSVファイル名をアルファベット順にソート
csv_files.sort()
print(csv_files)

['01アカマツ.csv', '02スギ.csv', '03オニグルミ.csv', '04イヌシデ.csv', '05コナラ.csv', '06クヌギ.csv', '07アカガシ.csv', '08ノイバラ.csv', '09カラスザンショウ.csv', '10アカメガシワ.csv', '11ヌルデ.csv', '12ゴンズイ.csv', '13ミズキ.csv', '14エゴノキ.csv', '15イボタノキ.csv', '16クサギ.csv', '17ガマズミ.csv', '18ニワトコ.csv', '19ツリガネニンジン.csv', '20カラスウリ.csv', '21タチツボスミレ.csv', '22メドハギ.csv', '23クズ.csv', '24ヘビイチゴ.csv', '25キンミズヒキ.csv', '26クサノオウ.csv', '27ムラサキケマン.csv', '28イヌタデ.csv', '29ミズヒキ.csv', '30ネジバナ.csv', '31シュンラン.csv', '32ノカンゾウ.csv', '33ノビル.csv', '34ウバユリ.csv', '35ヤブラン.csv', '36チカラシバ.csv', '37ホンドタヌキ0.csv', '38アブラコウモリ.csv', '39アズマモグラ.csv', '40カルガモ.csv', '41オナガ.csv', '42カワセミ.csv', '43コゲラ.csv', '44ハクセキレイ.csv', '45ツバメ.csv', '46スズメ.csv', '47キジバト.csv', '48ウグイス.csv', '49メジロ.csv', '50ニホンカナヘビ.csv', '51ニホントカゲ.csv', '52アオダイショウ.csv', '53ニホンヤモリ.csv', '54ニホンアカガエル0.csv', '55アズマヒキガエル.csv', '56ヒダリマキマイマイ.csv', '57ミスジマイマイ.csv', '58クロアゲハ.csv', '59クロオオアリ.csv', '60ゴマダラカマキリ.csv', '61ハグロトンボ.csv', '62ベニシジミ.csv', '63クマゼミ0.

In [5]:
#生き物番号を指定（１～６６から２つ選ぶ）.但し37(ホンドタヌキ), 54(ニホンアカガエル)，63（クマゼミ），64（ヒグラシ）は観測データが無いので選ばないでください．
ikimono_num1=1  #アカマツ
ikimono_num2=42 #カワセミ

# PDFファイルに地図を保存
with PdfPages('musashino_maps.pdf') as pdf:
    # 1つの図に2つの縦長のサブプロットを作成
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 15))

    # 最初の地図を描画
    plot_musashino_map(ikimono_num1, ax1)

    # 2番目の地図を描画
    plot_musashino_map(ikimono_num2, ax2)

    # レイアウトを調整して余白を最小限に
    plt.subplots_adjust(hspace=0.1, left=0, right=1, top=0.95, bottom=0)

    # PDFに保存
    pdf.savefig(fig)
    plt.close(fig)

print("PDFファイル 'musashino_maps.pdf' が正常に作成されました！")

# PDFファイルをダウンロード
from google.colab import files
files.download('musashino_maps.pdf')


<ipython-input-3-1fbd0e8574bb>:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  geo_musashino = geo_musashino.merge(species_summary, on='S_NAME', how='left').fillna(0)
<ipython-input-3-1fbd0e8574bb>:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  geo_musashino = geo_musashino.merge(species_summary, on='S_NAME', how='left').fillna(0)


PDFファイル 'musashino_maps.pdf' が正常に作成されました！


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>